In [107]:
import matplotlib.pyplot as plt

In [162]:
class Cell(object):
    """
    A representation of a game cell.
    """
    
    def __init__(self, position, cell_type, constraint):
        """
        Creates a game cell.
        
        :parameter position: the position of the cell.
        :type position: 2-dimensional tuple.
        :parameter cell_type: {'free', 'free_lighted', 'light', 'wall', 'constrained_wall'}.
        :type cell_type: string.
        :parameter constraint: number of lights that must be adjacent to the current cell.
        :type constraint: int.
        """
        self.position = position
        self.cell_type = cell_type
        self.constraint = constraint
        
    def set_cell_type(self, cell_type):
        """
        Changes the cell type.
        
        :parameter cell_type: {'free', 'free_lighted', 'light'}.
        :type cell_type: string.
        """
        self.cell_type = cell_type
        

In [167]:
class Board(object):
    """
    A representation of the game board.
    """
    
    def __init__(self, width, height, wall_positions):
        """
        Creates the game board.
        
        :parameter width: number of columns of the board.
        :type width: int.
        :parameter height: number of rows of the board.
        :type height: int.
        :parameter wall_positions: a list of wall positions.
        :type wall_positions: list of 3-dimensional tuple: (row, column, constraint).
        """
        self.board_width = width
        self.board_height = height
        self.board = []
        for i in range(self.board_height):
            self.board.append([])
            for j in range(self.board_width):
                self.board[i].append([])

        for i in range(width):
            for j in range(height):
                current_pos = (i, j)
                cell_type = 'free'
                constraint = None
                
                # Verify if current cell is a wall
                for wall_pos in wall_positions:
                    if i == wall_pos[0] and j == wall_pos[1]:
                        if wall_pos[2] is None:
                            cell_type = 'wall'
                        else:
                            cell_type = 'constrained_wall'
                            constraint = wall_pos[2]

                self.board[i][j] = Cell(current_pos, cell_type, constraint)
        
    def is_pos_valid(self, position):
        """
        Verifies if a given position respects the board dimensions.
        
        :parameter position: position that we want to check.
        :type position: 2-dimensional tuple.
        :rtype: bool.
        """
        if position[0] < 0 or position[0] >= self.board_width:
            return False
        elif position[1] < 0 or position[1] >= self.board_height:
            return False
        return True
    
    def illuminate_cells(self, light_position):
        """
        Illuminate cells based on the light position.
        
        :parameter light_position: the position of the light.
        :type initial_position: 2-dimensional tuple.
        """
        current_row = light_position[0] - 1
        while current_row >= 0:
            cell_type = self.board[current_row][light_position[1]].cell_type
            if cell_type == 'free':
                self.board[current_row][light_position[1]].set_cell_type('free_lighted')
            elif cell_type == 'wall' or cell_type == 'constrained_wall':
                break
            current_row -= 1

        current_row = light_position[0] + 1
        while current_row < self.board_width:
            cell_type = self.board[current_row][light_position[1]].cell_type
            if cell_type == 'free':
                self.board[current_row][light_position[1]].set_cell_type('free_lighted')
            elif cell_type == 'wall' or cell_type == 'constrained_wall':
                break
            current_row += 1

        current_col = light_position[1] - 1
        while current_col >= 0:
            cell_type = self.board[light_position[0]][current_col].cell_type
            if cell_type == 'free':
                self.board[light_position[0]][current_col].set_cell_type('free_lighted')
            elif cell_type == 'wall' or cell_type == 'constrained_wall':
                break
            current_col -= 1

        current_col = light_position[1] + 1
        while current_col < self.board_height:
            cell_type = self.board[light_position[0]][current_col].cell_type
            if cell_type == 'free':
                self.board[light_position[0]][current_col].set_cell_type('free_lighted')
            elif cell_type == 'wall' or cell_type == 'constrained_wall':
                break
            current_col += 1
        
    def place_light(self, position):
        """
        Places a light at position if possible.
        
        :parameter position: the light's position.
        :type position: 2-dimensional tuple.
        :return: if the light was successfully placed at position.
        :rtype: bool.
        """
        if self.is_pos_valid(position):
            if self.board[position[0]][position[1]].cell_type == 'free' or self.board[position[0]][position[1]].cell_type == 'free_lighted': 
                self.board[position[0]][position[1]] = Cell(position, 'light', None)
                self.illuminate_cells(position)
                                
                return True
            else:
                return False
        else:
            raise IndexError()
    
    def print_ascii_board(self):
        """
        Prints board using ASCII characters.
        F character is used for free cells.
        W character is used for unconstrained walls.
        L character is used for lights.
        FL string is used for free lighted cells.
        Numbers are used for constrained walls. They indicate the number of adjacent lights required.
        """
        for i in range(self.board_width):
            for j in range(self.board_height):
                cell_type = self.board[i][j].cell_type
                if cell_type == 'free':
                    print('F  ', end='')
                elif cell_type == 'constrained_wall':
                    print(f'{self.board[i][j].constraint}  ', end='')
                elif cell_type == 'wall':
                    print('W  ', end='')
                elif cell_type == 'light':
                    print('L  ', end='')
                elif cell_type == 'free_lighted':
                    print('FL ', end='')
            print('\n')

In [168]:
width = 7
height = 7
wall_positions = [(0, 2, 1), 
                  (1, 3, 0), 
                  (1, 4, None), 
                  (2, 1, 0), 
                  (2, 6, 2), 
                  (3, 1, 0), 
                  (3, 3, None), 
                  (3, 5, None),
                  (4, 0, None),
                  (4, 5, 1),
                  (5, 2, 3),
                  (5, 3, 1),
                  (6, 4, None)]

board = Board(width, height, wall_positions)

In [169]:
board.print_ascii_board()

F  F  1  F  F  F  F  

F  F  F  0  W  F  F  

F  0  F  F  F  F  2  

F  0  F  W  F  W  F  

W  F  F  F  F  1  F  

F  F  3  1  F  F  F  

F  F  F  F  W  F  F  



In [170]:
board.place_light((4, 1))

True

In [171]:
board.print_ascii_board()

F  F  1  F  F  F  F  

F  F  F  0  W  F  F  

F  0  F  F  F  F  2  

F  0  F  W  F  W  F  

W  L  FL FL FL 1  F  

F  FL 3  1  F  F  F  

F  FL F  F  W  F  F  



In [172]:
board.place_light((3, 3))

False

In [173]:
board.place_light((4, 4))

True

In [174]:
board.print_ascii_board()

F  F  1  F  F  F  F  

F  F  F  0  W  F  F  

F  0  F  F  FL F  2  

F  0  F  W  FL W  F  

W  L  FL FL L  1  F  

F  FL 3  1  FL F  F  

F  FL F  F  W  F  F  



In [175]:
board.place_light((2, 1))

False

In [176]:
board.place_light((1, 1))

True

In [177]:
board.print_ascii_board()

F  FL 1  F  F  F  F  

FL L  FL 0  W  F  F  

F  0  F  F  FL F  2  

F  0  F  W  FL W  F  

W  L  FL FL L  1  F  

F  FL 3  1  FL F  F  

F  FL F  F  W  F  F  

